# Prepare dataset

In [ ]:
# download, decompress the data
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip 
!unzip balloon_dataset.zip
!rm -r __MACOSX

In [1]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session() # can use LocalSession() to run container locally

bucket = sess.default_bucket()
region = "us-east-2"
prefix_input = 'detectron2-input'
prefix_output = 'detectron2-ouput'

In [15]:
# now need to upload it to S3

import os
import boto3

s3_resource = boto3.resource("s3", region_name=region)
root_path = os.getcwd()
data_path = os.path.join(root_path, "balloon")

for path, subdirs, files in os.walk(data_path):
    directory_name = path.replace(root_path+"/","")
    for file in files:
        #print(os.path.join(root_path, directory_name, file))
        s3_resource.Bucket(bucket).upload_file(os.path.join(root_path, directory_name, file), directory_name+'/'+file)
        #print(f"file {file} has been uploaded to S3")

# Train model

Let's review the training script.

In [2]:
! pygmentize d2_script/train.py

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random

import subprocess
import sys
    
# install pycocotools
subprocess.check_call([sys.executable, "-m", "pip", "install", "git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"])
    
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultTrainer


# packages neededs for custom dataset
import os
import numpy as np
import json
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog
    

def train():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = ("ball

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

role = get_execution_role()

d2 = PyTorch(entry_point="train.py", source_dir="d2_script",
             train_instance_count=1,
             role=role,
             train_instance_type='ml.p3.2xlarge',
             framework_version="1.4.0",
             debugger_hook_config=False)

#d2.set_hyperparameters(num_epochs = 1, num_classes = 2, )

d2.fit({'train':"s3://sagemaker-us-east-2-553020858742/balloon/train",
        'val':"s3://sagemaker-us-east-2-553020858742/balloon/val"})

2020-04-07 16:44:47 Starting - Starting the training job...
2020-04-07 16:44:48 Starting - Launching requested ML instances...
2020-04-07 16:45:46 Starting - Preparing the instances for training.........
2020-04-07 16:47:07 Downloading - Downloading input data
2020-04-07 16:47:07 Training - Downloading the training image.........
2020-04-07 16:48:44 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-07 16:48:45,375 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-04-07 16:48:45,399 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-04-07 16:48:48,425 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-04-07 16:48:48,717 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2020-04-0